# Processing GMOS-N and GMOS-S science data

Performs overscan correction, trimming, bias subtraction, flat-field correction, and cosmic ray removal on raw Gemini science data downloaded from the Gemini archive.  Uses raw bias data downloaded from the Gemini archive and processed flatfield files created by <tt>notebook_GMOS_a01_12amp_process_raw_twilight_flats.ipynb</tt> and <tt>notebook_GMOS_a02_12amp_combine_twilight_flatfield_images.ipynb</tt>.

## Before running notebook:

- Install <tt>fpack</tt> and <tt>funpack</tt> if not already installed (https://heasarc.gsfc.nasa.gov/fitsio/fpack/); change paths to local installation location in functions <tt>compress_file_fpack</tt> and <tt>decompress_file_fpack</tt> in this notebook<br><br>

- Requires <tt>cosmics_py3.py</tt> to be present in same directory as notebook; uses the L.A.Cosmic cosmic ray removal algorithm developed by Pieter van Dokkum (https://arxiv.org/abs/astro-ph/0108003)<br><br>

- If not already done, run <tt>notebook_GMOS_a01_12amp_process_raw_twilight_flats.ipynb</tt> and <tt>notebook_GMOS_a02_12amp_combine_twilight_flatfield_images</tt> to prepare flatfield images closest to the given night; place these files in the main directory where filenames should follow the format of <tt>nYYYYMMDD.instr.twiskyflat.?.chipN.fits.fz</tt>, where "<tt>YYYYMMDD</tt>" is the date associated with the median-combined flatfield data, "<tt>instr</tt>" indicates the instrument used ("<tt>gmosn</tt>" for GMOS-N and "<tt>gmoss</tt>" for GMOS-S), "<tt>?</tt>" is the filter associated with that flatfield file, and "<tt>N</tt>" is the chip number of that flatfield file (i.e., 1, 2, or 3).<br><br>

- Download science data for a particular night from https://archive.gemini.edu/searchform; unpack archive, separate individual observations by filter, and place data files into subdirectories named <tt>rawfits_science_?</tt> in the main data directory where "<tt>?</tt>" is the filter associated with that subdirectory<br><br>

- Go to https://archive.gemini.edu/searchform; specify UTC Date of corresponding flatfield data, "GMOS-N" or "GMOS-S" for Instrument, "BIAS" as Obs. Type, "2x2" for Binning, and "Full Frame" for ROI; search for bias frames corresponding to each night of flatfield data; download data and place in a subdirectory named <tt>rawfits_bias</tt> in the main data directory<br><br>

- Run <tt>process_gemini_science_directory(data_path,observatory,date_id)</tt> where <tt>data_path</tt> is the main data directory, observatory is either "<tt>GeminiN</tt>" or "<tt>GeminiS</tt>, and <tt>date_id</tt> is the date of observations in YYYYMMDD format


In [ ]:
import os, sys
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.modeling import models
from astropy import nddata
from astropy import units as u
import ccdproc as cp
from ccdproc import CCDData
import numpy as np
import math, glob, subprocess
import cosmics_py3
import datetime


In [ ]:
def create_directory(path):
    if not os.path.exists(path):
        os.mkdir(path)
    else:
        print('Directory {:s} already exists.'.format(path))
    return None

def decompress_file_bzip2(filename):
    cmd = ['bzip2','-d',filename]
    process = subprocess.call(cmd)
    return None

def compress_file_fpack(filename):
    process = subprocess.call(['/Users/hhsieh/Astro/tools/cfitsio/fpack',filename])
    os.remove(filename)
    return None

def decompress_file_fpack(filename):
    process = subprocess.call(['/Users/hhsieh/Astro/tools/cfitsio/funpack',filename])
    os.remove(filename)
    return None

def decompress_directory_bzip2(file_path):
    print('>>> Starting decompression of bz2 files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for bz2_file in sorted(glob.glob('*.bz2')):
        decompress_file_bzip2(bz2_file)
    print('>>> Decompression of bz2 files complete.')
    return None

def compress_directory_fpack(file_path):
    print('>>> Starting fpack compression of FITS files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fits_file in glob.glob('*.fits'):
        compress_file_fpack(fits_file)
    print('>>> fpack compression of FITS files complete.')
    return None
    
def decompress_directory_fpack(file_path):
    print('>>> Starting decompression of fz files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fz_file in glob.glob('*.fz'):
        decompress_file_fpack(fz_file)
    print('>>> Decompression of fz files complete.')
    return None


In [ ]:
def get_date_id():
    # obtain YYYYMMDD date from filename of GMOS FITS files being processed (regardless of current compression state)
    fits_date = ''
    if len(glob.glob('*.fits')) > 0:
        for raw_mef_file in sorted(glob.glob('*.fits')):
            fits_date = raw_mef_file[1:9]
    elif len(glob.glob('*.fits.fz')) > 0:
        for raw_mef_file in sorted(glob.glob('*.fits.fz')):
            fits_date = raw_mef_file[1:9]
    elif len(glob.glob('*.fits.bz2')) > 0:
        for raw_mef_file in sorted(glob.glob('*.fits.bz2')):
            fits_date = raw_mef_file[1:9]
    return fits_date

def create_stats_files(fits_date,imgtype):
    for extid in range(2,10):
        ext_stats = 'n{:s}.{:s}.{:02d}.stats'.format(fits_date,imgtype,extid+1)
        with open(ext_stats,'w') as of:
            of.write('# Extension {:02d}              NPIX        MEAN     STDDEV         MIN         MAX\n'.format(extid+1))
    return None
        
def compile_stats_files(fits_date,imgtype):
    print('Starting image statistics output...')
    output_stats_filename = 'n{:s}.{:s}.stats'.format(fits_date,imgtype)
    with open(output_stats_filename,'w') as output_stats_file:
        for stats_filename in glob.glob('*.??.stats'):
            with open(stats_filename,'r') as stats_file:
                for line in stats_file:
                    output_stats_file.write(line)
            os.remove(stats_filename)
    print('>>> Image statistics output complete.')
    return None

def split_extensions(imgtype):
    # split 12-element GMOS multi-extension FITS files into individual elements
    # return: dimensions of extensions; also writes individual extension files to working directory
    for raw_mef_file in glob.glob('*.fits'):
        with fits.open(raw_mef_file) as hdulist:
            print('Splitting {:s}...'.format(raw_mef_file))
            fits_date = raw_mef_file[1:9]
            fits_id   = raw_mef_file[11:14]
            hdr1 = getheader(raw_mef_file,0)
            for extid in range(2,10):
                extension = hdulist[extid+1].data
                hdr2 = hdulist[extid+1].header
                radecsys = hdr2['RADECSYS']
                hdr2['RADESYSA'] = radecsys
                del hdr2['RADECSYS']
                imstat_npix = extension.size
                imstat_min,imstat_max    = np.min(extension),np.max(extension)
                imstat_mean,imstat_stdev = np.mean(extension),np.std(extension)
                ext_filename = 'n' + fits_date + '.' + fits_id + '.{:02d}.fits'.format(extid+1)
                outputfilename = 'n{:s}.{:s}.{:02d}.stats'.format(fits_date,imgtype,extid+1)
                with open(outputfilename,'a') as of:
                    of.write('{:s}   {:>8d}    {:>8.2f}    {:>7.2f}    {:>8.2f}    {:>8.2f}\n'.format(ext_filename,imstat_npix,imstat_mean,imstat_stdev,imstat_min,imstat_max))
                dimensions = extension.shape
                dimension1 = dimensions[0]
                dimension2 = dimensions[1]
                hdr = hdr1 + hdr2
                fits.writeto(ext_filename,extension,hdr)
    return dimension1, dimension2
    print('>>> Multi-extension fits file splitting complete.')
    return None


def add_fits_header_line(fits_file,param_name,param_value,param_comment):
    # Add line to FITS header
    with fits.open(fits_file) as hdulist:
        data = hdulist[0].data
        hdr = hdulist[0].header
    hdr[param_name] = (param_value,param_comment)
    fits.writeto(fits_file,data,hdr,overwrite=True)
    return None

def overscan_and_trim():
    # Do overscan correction and trim images in current directory
    print('Starting overscan correction and trimming...')
    for fits_file in sorted(glob.glob('*.???.??.fits')):
        ot_fits_file = fits_file[0:16] + '.ot.fits'
        fits_data = CCDData.read(fits_file,unit=u.adu)
        file_ext = fits_file[14:16]
        # Overscan correction
        if ((file_ext == '03') or (file_ext=='05') or (file_ext=='07') or (file_ext=='09')):
            o_fits_data  = cp.subtract_overscan(fits_data, fits_section='[257:288,1:2112]', overscan_axis=1, add_keyword={'oscansub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        else:
            o_fits_data  = cp.subtract_overscan(fits_data, fits_section='[1:32,1:2112]', overscan_axis=1, add_keyword={'oscansub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        # Image trimming
        if ((file_ext == '03') or (file_ext=='07')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[1:256,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        elif ((file_ext == '04') or (file_ext == '08')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[33:282,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        elif ((file_ext == '05') or (file_ext =='09')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[7:256,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        else: # file_ext == '02' or file_ext == '10'
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[33:288,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        ot_fits_data.write(ot_fits_file)
        os.remove(fits_file)
    print('>>> Overscan correction and trimming complete.')
    return None

def bias_median_combine(dateid):
    print('Starting median combination of bias frames...')
    for extid in range(2,10):
        bias_list = []
        ext_file_format = '*.{:02d}.ot.fits'.format(extid+1)
        output_filename = 'n' + dateid + '.bias.{:02d}.fits'.format(extid+1)
        for fits_file in sorted(glob.glob(ext_file_format)):
            fits_data = CCDData.read(fits_file)
            bias_list.append(fits_data)
            os.remove(fits_file)
        master_bias = cp.combine(bias_list,method='median')
        master_bias.write(output_filename)
    print('>>> Median combination of bias frames complete.')
    return None

def bias_correct(dateid,cwd_raw_bias):
    print('Starting bias subtraction...')
    for extid in range(2,10):
        bias_filename = cwd_raw_bias + 'n' + dateid + '.bias.{:02d}.fits'.format(extid+1)
        ext_file_format = '*.{:02d}.ot.fits'.format(extid+1)
        for fits_file in sorted(glob.glob(ext_file_format)):
            fits_data = CCDData.read(fits_file)
            bias_data = CCDData.read(bias_filename)
            fits_date_imageid = fits_file[0:13]
            output_filename = fits_date_imageid + '.{:02d}.otz.fits'.format(extid+1)
            bias_corrected_data = cp.subtract_bias(fits_data,bias_data,add_keyword={'zerocorr':True,'calstat':'OTZ'})
            bias_corrected_data.write(output_filename)
            os.remove(fits_file)
    print('>>> Bias subtraction complete.')
    return None

def concatenate_gmos_amps():
    print('>>> Starting adjacent amp area concatenation...')
    for fits_file in sorted(glob.glob('*.03.otz.fits')):
        file_prefix = fits_file[0:13]
        ext1_filename = file_prefix + '.03.otz.fits'
        ext2_filename = file_prefix + '.04.otz.fits'
        hdr = getheader(ext1_filename,0)
        with fits.open(ext1_filename) as hdul1, fits.open(ext2_filename) as hdul2:
            ext1_data = hdul1[0].data
            ext2_data = hdul2[0].data
        chip1_data = np.hstack((ext1_data,ext2_data))
        output_filename = file_prefix + '.chip1.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        os.remove(ext1_filename)
        os.remove(ext2_filename)

        ext1_filename = file_prefix + '.05.otz.fits'
        ext2_filename = file_prefix + '.06.otz.fits'
        ext3_filename = file_prefix + '.07.otz.fits'
        ext4_filename = file_prefix + '.08.otz.fits'
        hdr = getheader(ext1_filename,0)
        with fits.open(ext1_filename) as hdul1, fits.open(ext2_filename) as hdul2, fits.open(ext3_filename) as hdul3, fits.open(ext4_filename) as hdul4:        
            ext1_data = hdul1[0].data
            ext2_data = hdul2[0].data
            ext3_data = hdul3[0].data
            ext4_data = hdul4[0].data
        chip1_data = np.hstack((ext1_data,ext2_data,ext3_data,ext4_data))
        output_filename = file_prefix + '.chip2.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        os.remove(ext1_filename)
        os.remove(ext2_filename)
        os.remove(ext3_filename)
        os.remove(ext4_filename)

        ext1_filename = file_prefix + '.09.otz.fits'
        ext2_filename = file_prefix + '.10.otz.fits'
        hdr = getheader(ext1_filename,0)
        with fits.open(ext1_filename) as hdul1, fits.open(ext2_filename) as hdul2:
            ext1_data = hdul1[0].data
            ext2_data = hdul2[0].data
        chip1_data = np.hstack((ext1_data,ext2_data))
        output_filename = file_prefix + '.chip3.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        os.remove(ext1_filename)
        os.remove(ext2_filename)
    print('>>> Adjacent amp area concatenation complete.')
    return None

def flatfield_correction(data_path,flatfield_chip1_file,flatfield_chip2_file,flatfield_chip3_file):
    print('Starting flatfield correction...')
    for fits_file in sorted(glob.glob('*.chip1.otz.fits')):
        fits_data = CCDData.read(fits_file)
        flat_data = CCDData.read(data_path+flatfield_chip1_file,unit=u.adu)
        fits_date_imageid = fits_file[0:13]
        output_filename = fits_date_imageid + '.chip1.otzf.fits'
        flat_corrected_data = cp.flat_correct(fits_data,flat_data,add_keyword={'flatcorr': True, 'calstat': 'OTZF'})
        flat_corrected_data.write(output_filename)
        add_fits_header_line(output_filename,'BIAS_1',fits_date_imageid+'.bias.03.fits','1st bias frame used')
        add_fits_header_line(output_filename,'BIAS_2',fits_date_imageid+'.bias.04.fits','2nd bias frame used')
        add_fits_header_line(output_filename,'BIAS_3','n/a','3rd bias frame used')
        add_fits_header_line(output_filename,'BIAS_4','n/a','4th bias frame used')
        add_fits_header_line(output_filename,'FLATUSED',flatfield_chip1_file,'Flatfield frame used')
        os.remove(fits_file)

    for fits_file in sorted(glob.glob('*.chip2.otz.fits')):
        fits_data = CCDData.read(fits_file)
        flat_data = CCDData.read(data_path+flatfield_chip2_file,unit=u.adu)
        fits_date_imageid = fits_file[0:13]
        output_filename = fits_date_imageid + '.chip2.otzf.fits'
        flat_corrected_data = cp.flat_correct(fits_data,flat_data,add_keyword={'flatcorr': True, 'calstat': 'OTZF'})
        flat_corrected_data.write(output_filename)
        add_fits_header_line(output_filename,'BIAS_1',fits_date_imageid+'.bias.05.fits','1st bias frame used')
        add_fits_header_line(output_filename,'BIAS_2',fits_date_imageid+'.bias.06.fits','2nd bias frame used')
        add_fits_header_line(output_filename,'BIAS_3',fits_date_imageid+'.bias.07.fits','3rd bias frame used')
        add_fits_header_line(output_filename,'BIAS_4',fits_date_imageid+'.bias.08.fits','4th bias frame used')
        add_fits_header_line(output_filename,'FLATUSED',flatfield_chip2_file,'Flatfield frame used')
        os.remove(fits_file)

    for fits_file in sorted(glob.glob('*.chip3.otz.fits')):
        fits_data = CCDData.read(fits_file)
        flat_data = CCDData.read(data_path+flatfield_chip3_file,unit=u.adu)
        fits_date_imageid = fits_file[0:13]
        output_filename = fits_date_imageid + '.chip3.otzf.fits'
        flat_corrected_data = cp.flat_correct(fits_data,flat_data,add_keyword={'flatcorr': True, 'calstat': 'OTZF'})
        flat_corrected_data.write(output_filename)
        add_fits_header_line(output_filename,'BIAS_1',fits_date_imageid+'.bias.09.fits','1st bias frame used')
        add_fits_header_line(output_filename,'BIAS_2',fits_date_imageid+'.bias.10.fits','2nd bias frame used')
        add_fits_header_line(output_filename,'BIAS_3','n/a','3rd bias frame used')
        add_fits_header_line(output_filename,'BIAS_4','n/a','4th bias frame used')
        add_fits_header_line(output_filename,'FLATUSED',flatfield_chip3_file,'Flatfield frame used')
        os.remove(fits_file)

    print('>>> Flatfield correction complete.')
    return None

def cosmicray_cleaning():
    print('Starting cosmic ray correction...')
    for fits_file in sorted(glob.glob('*.chip1.otzf.fits')):
        print('Cosmic ray cleaning for {:s}'.format(fits_file))
        file_prefix = fits_file[0:19]
        hdr = getheader(fits_file,0)
        fits_gain = hdr['gain']
        fits_readnoise = hdr['rdnoise']
        array,header = cosmics_py3.fromfits(fits_file,verbose=False)
        c = cosmics_py3.cosmicsimage(array,gain=fits_gain,readnoise=fits_readnoise,sigclip=5.0,sigfrac=0.3,objlim=5.0,verbose=False)
        c.run(maxiter=4)
        output_file = file_prefix + '.otzfc.fits'
        cosmics_py3.tofits(output_file,c.cleanarray,header,verbose=False)
        add_fits_header_line(output_file,'CRCORR',True,'Cosmic ray removal performed? (T/F)')
        add_fits_header_line(output_file,'CALSTAT','OTZFC','Image calibration status')
        add_fits_header_line(output_file,'CRR_GAIN',fits_gain,'CR removal parameter: gain')
        add_fits_header_line(output_file,'CRR_NOIS',fits_readnoise,'CR removal parameter: read noise')
        add_fits_header_line(output_file,'CRR_SIGC',5.0,'CR removal parameter: sigclip')
        add_fits_header_line(output_file,'CRR_SIGF',0.3,'CR removal parameter: sigfrac')
        add_fits_header_line(output_file,'CRR_OBJL',5.0,'CR removal parameter: objlim')
        os.remove(fits_file)

    for fits_file in sorted(glob.glob('*.chip2.otzf.fits')):
        print('Cosmic ray cleaning for {:s}'.format(fits_file))
        file_prefix = fits_file[0:19]
        hdr = getheader(fits_file,0)
        fits_gain = hdr['gain']
        fits_readnoise = hdr['rdnoise']
        array,header = cosmics_py3.fromfits(fits_file,verbose=False)
        c = cosmics_py3.cosmicsimage(array,gain=fits_gain,readnoise=fits_readnoise,sigclip=5.0,sigfrac=0.3,objlim=5.0,verbose=False)
        c.run(maxiter=4)
        output_file = file_prefix + '.otzfc.fits'
        cosmics_py3.tofits(output_file,c.cleanarray,header,verbose=False)
        add_fits_header_line(output_file,'CRCORR',True,'Cosmic ray removal performed? (T/F)')
        add_fits_header_line(output_file,'CALSTAT','OTZFC','Image calibration status')
        add_fits_header_line(output_file,'CRR_GAIN',fits_gain,'CR removal parameter: gain')
        add_fits_header_line(output_file,'CRR_NOIS',fits_readnoise,'CR removal parameter: read noise')
        add_fits_header_line(output_file,'CRR_SIGC',5.0,'CR removal parameter: sigclip')
        add_fits_header_line(output_file,'CRR_SIGF',0.3,'CR removal parameter: sigfrac')
        add_fits_header_line(output_file,'CRR_OBJL',5.0,'CR removal parameter: objlim')
        os.remove(fits_file)

    for fits_file in sorted(glob.glob('*.chip3.otzf.fits')):
        print('Cosmic ray cleaning for {:s}'.format(fits_file))
        file_prefix = fits_file[0:19]
        hdr = getheader(fits_file,0)
        fits_gain = hdr['gain']
        fits_readnoise = hdr['rdnoise']
        array,header = cosmics_py3.fromfits(fits_file,verbose=False)
        c = cosmics_py3.cosmicsimage(array,gain=fits_gain,readnoise=fits_readnoise,sigclip=5.0,sigfrac=0.3,objlim=5.0,verbose=False)
        c.run(maxiter=4)
        output_file = file_prefix + '.otzfc.fits'
        cosmics_py3.tofits(output_file,c.cleanarray,header,verbose=False)
        add_fits_header_line(output_file,'CRCORR',True,'Cosmic ray removal performed? (T/F)')
        add_fits_header_line(output_file,'CALSTAT','OTZFC','Image calibration status')
        add_fits_header_line(output_file,'CRR_GAIN',fits_gain,'CR removal parameter: gain')
        add_fits_header_line(output_file,'CRR_NOIS',fits_readnoise,'CR removal parameter: read noise')
        add_fits_header_line(output_file,'CRR_SIGC',5.0,'CR removal parameter: sigclip')
        add_fits_header_line(output_file,'CRR_SIGF',0.3,'CR removal parameter: sigfrac')
        add_fits_header_line(output_file,'CRR_OBJL',5.0,'CR removal parameter: objlim')
        os.remove(fits_file)
        
    print('>>> Cosmic ray correction complete.')
    return None

def move_reduced_data(rawfitsdir,rawbiasdir,flatdir,path_stats):
    os.chdir(rawfitsdir)
    for fits_file in sorted(glob.glob('*.otzfc.fits')):
        os.rename(rawfitsdir+fits_file,flatdir+fits_file)
    for stats_file in sorted(glob.glob('*.stats')):
        os.rename(rawfitsdir+stats_file,path_stats+stats_file)
    os.chdir(rawbiasdir)
    for fits_file in sorted(glob.glob('*.bias.??.fits')):
        os.rename(rawbiasdir+fits_file,flatdir+fits_file)
    for stats_file in sorted(glob.glob('*.stats')):
        os.rename(rawbiasdir+stats_file,path_stats+stats_file)
    print('>>> Processed data moved to output directory.')
    return None

def move_reduced_science_data(rawfitsdir,flatdir,path_stats):
    #move_file = 'mv'
    os.chdir(rawfitsdir)
    for fits_file in sorted(glob.glob('*.otzfc.fits')):
        os.rename(rawfitsdir+fits_file,flatdir+fits_file)
        #move_cmd = [move_file,fits_file,flatdir]
        #process = subprocess.call(move_cmd)
    for stats_file in sorted(glob.glob('*.stats')):
        os.rename(rawfitsdir+stats_file,path_stats+stats_file)
        #move_cmd = [move_file,stats_file,path_stats]
        #process = subprocess.call(move_cmd)
    print('>>> Processed data moved to output directory.')
    return None

def move_reduced_bias_data(rawbiasdir,flatdir,path_stats):
    #move_file = 'mv'
    os.chdir(rawbiasdir)
    if os.path.exists(flatdir):
        for fits_file in sorted(glob.glob('*.bias.??.fits')):
            os.rename(rawbiasdir+fits_file,flatdir+fits_file)
            #move_cmd = [move_file,fits_file,flatdir]
            #process = subprocess.call(move_cmd)
    for stats_file in sorted(glob.glob('*.stats')):
        os.rename(rawbiasdir+stats_file,path_stats+stats_file)
        #move_cmd = [move_file,stats_file,path_stats]
        #process = subprocess.call(move_cmd)
    print('>>> Processed bias data moved to output directory.')
    return None

#def move_flatfield_images(base_dir,data_dir,path_procfits,flatfield_chip1,flatfield_chip2,flatfield_chip3):
#    os.chdir(base_dir + data_dir)
#    os.rename(base_dir+data_dir+flatfield_chip1,path_procfits+flatfield_chip1)
#    os.rename(base_dir+data_dir+flatfield_chip2,path_procfits+flatfield_chip2)
#    os.rename(base_dir+data_dir+flatfield_chip3,path_procfits+flatfield_chip3)
#    return None


In [ ]:
def fix_gain_rdnoise_gmosn_hamamatsu():
    # Fix gain and read noise values in FITS headers
    for fits_file in sorted(glob.glob('*.???.??.fits')):
        with fits.open(fits_file) as hdulist:
            data = hdulist[0].data
            hdr = hdulist[0].header
        hdr['GAIN'] = 1.63
        hdr['RDNOISE'] = 4.14
        fits.writeto(fits_file,data,hdr,overwrite=True)
    print('>>> Gain and read noise updated in image headers.')
    return None

def fix_gain_rdnoise_gmoss_hamamatsu():
    # Fix gain and read noise values in FITS headers
    for fits_file in sorted(glob.glob('*.???.??.fits')):
        with fits.open(fits_file) as hdulist:
            data = hdulist[0].data
            hdr = hdulist[0].header
        hdr['GAIN'] = 1.83
        hdr['RDNOISE'] = 3.98
        fits.writeto(fits_file,data,hdr,overwrite=True)
    print('>>> Gain and read noise updated in image headers.')
    return None

def fix_gmos_header_info(instr_id,filter_id):
    # Fix header information for PDS submission
    for fits_file in sorted(glob.glob('*.otzfc.fits')):
        with fits.open(fits_file) as hdulist:
            hdr,data = hdulist[0].header,hdulist[0].data

        # Change keywords for CDi_j parameters (conflict with other WCS keywords)
        cd1_1,cd1_1_comment = hdr['CD1_1'],hdr.comments['CD1_1']
        cd1_2,cd1_2_comment = hdr['CD1_2'],hdr.comments['CD1_2']
        cd2_1,cd2_1_comment = hdr['CD2_1'],hdr.comments['CD2_1']
        cd2_2,cd2_2_comment = hdr['CD2_2'],hdr.comments['CD2_2']
        hdr['x_CD1_1'] = (cd1_1,cd1_1_comment)
        hdr['x_CD1_2'] = (cd1_2,cd1_2_comment)
        hdr['x_CD2_1'] = (cd2_1,cd2_1_comment)
        hdr['x_CD2_2'] = (cd2_2,cd2_2_comment)
        del hdr['CD1_1']
        del hdr['CD1_2']
        del hdr['CD2_1']
        del hdr['CD2_2']
        
        # Change keyword for EPOCH parameter
        target_epoch,target_epoch_comment = hdr['EPOCH'],hdr.comments['EPOCH']
        hdr['TRGEPOCH'] = (target_epoch,target_epoch_comment)
        del hdr['EPOCH']
        
        # Remove RADVEL parameter (not correctly populated)
        del hdr['RADVEL']
        
        # Change EXPTIME value to integer format
        exptime = int(round(hdr['EXPTIME']))
        hdr['EXPTIME'] = exptime
        
        # Add central filter wavelength keyword
        if filter_id == 'g':   hdr['CENTWAVE'] = 475
        elif filter_id == 'r': hdr['CENTWAVE'] = 630
        elif filter_id == 'i': hdr['CENTWAVE'] = 780
        elif filter_id == 'z': hdr['CENTWAVE'] = 876
        
        # Remove checksums
        if 'CHECKSUM' in hdr: del hdr['CHECKSUM']
        if 'DATASUM'  in hdr: del hdr['DATASUM']
        
        fits.writeto(fits_file,data,hdr,overwrite=True,checksum=True)
        
    print('>>> Header information fixed for PDS submission.')
    return None

def find_flatfield_files(data_path,instr_id,filter_id):
    ff_chip1_filename,ff_chip2_filename,ff_chip3_filename = 'none','none','none'
    os.chdir(data_path)
    decompress_directory_fpack(data_path)
    flatfield_filename_pattern = 'n*.' + instr_id + '.twiskyflat.' + filter_id + '*.fits'
    if len(glob.glob(flatfield_filename_pattern)) != 3:
        print('Incorrect number of flatfield files found.')
    else:
        for flatfield_file in sorted(glob.glob(flatfield_filename_pattern)):
            if flatfield_file[-10:-5] == 'chip1': ff_chip1_filename = flatfield_file
            if flatfield_file[-10:-5] == 'chip2': ff_chip2_filename = flatfield_file
            if flatfield_file[-10:-5] == 'chip3': ff_chip3_filename = flatfield_file
    return ff_chip1_filename,ff_chip2_filename,ff_chip3_filename
                
print('Done.')


In [ ]:
def process_bias_data(path_rawbias,instr_id,date_id,reduction_log):
    print('\n{:s} - Processing bias-frame data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawbias))
    reduction_log.write('\n{:s} - Processing bias-frame science data in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawbias))
    
    # decompress raw bias fits files
    print('\n{:s} - Decompressing, filtering, and splitting bias files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    reduction_log.write('{:s} - Decompressing, filtering, and splitting bias files...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    os.chdir(path_rawbias)
    decompress_directory_bzip2(path_rawbias)       # decompress downloaded GMOS files
    decompress_directory_fpack(path_rawbias)       # decompress previously fpacked GMOS files
    #date_id = get_date_id()
    create_stats_files(date_id,'bias')       # create files to record image statistics
    split_extensions('bias')                 # split bias MEFs into individual FITS files
    if instr_id == 'gmosn':
        fix_gain_rdnoise_gmosn_hamamatsu()
    elif instr_id == 'gmoss':
        fix_gain_rdnoise_gmoss_hamamatsu()
    compile_stats_files(date_id,'bias')      # collect image stats of bias files together and delete individual files
    
    # perform overscan corrections and trim data for bias frames
    os.chdir(path_rawbias)
    overscan_and_trim()
    bias_median_combine(date_id)
    
    print('\n{:s} - Processing bias-frame data in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawbias))
    reduction_log.write('\n{:s} - Processing bias-frame science data in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawbias))
    return reduction_log


def process_science_data(data_path,path_rawfits,path_rawbias,path_procfits,path_stats,instr_id,filter_id,reduction_log):
    print('\n{:s} - Processing {:s}-band science data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),filter_id,data_path))
    reduction_log.write('\n{:s} - Processing {:s}-band science data in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),filter_id,data_path))
    
    flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename = find_flatfield_files(data_path,instr_id,filter_id)
    
    if flatfield_chip1_filename != 'none' and flatfield_chip2_filename != 'none' and flatfield_chip3_filename != 'none':
        if not os.path.isdir(path_procfits): os.mkdir(path_procfits)
        if not os.path.isdir(path_stats):    os.mkdir(path_stats)
        # Process raw science fits files
        print('\n{:s} - Decompressing and splitting data files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
        reduction_log.write('{:s} - Decompressing and splitting data files...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
        os.chdir(path_rawfits)
        decompress_directory_bzip2(path_rawfits)       # decompress downloaded GMOS files
        decompress_directory_fpack(path_rawfits)       # decompress previously fpacked GMOS files
        date_id = get_date_id()
        create_stats_files(date_id,'scidata.{:s}'.format(filter_id))    # create files to record image statistics
        dim1, dim2 = split_extensions('scidata') # split science MEFs into individual FITS files
        if instr_id == 'gmosn':
            fix_gain_rdnoise_gmosn_hamamatsu()
        elif instr_id == 'gmoss':
            fix_gain_rdnoise_gmoss_hamamatsu()
        compile_stats_files(date_id,'scidata.{:s}'.format(filter_id))   # collect image stats of science files together and delete individual files
            
        # Process science frames
        os.chdir(path_rawfits)
        overscan_and_trim()                                                    # Perform overscan correction and trim data
        bias_correct(date_id,path_rawbias)                                      # Perform bias correction
        concatenate_gmos_amps()                                                # Join science frames
        flatfield_correction(data_path,flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename)  # Do flatfield corrections
        cosmicray_cleaning()                                                   # Do cosmic ray cleaning
    
        print('{:s} - Processing science data in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
        reduction_log.write('{:s} - Processing science data in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
            
    else:
        print('{:s} - Appropriate flatfield images for {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawfits))        
        reduction_log.write('{:s} - Appropriate flatfield images for {:s} not found.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawfits))        
            
    return reduction_log


def clean_up_science_data(path_rawfits,path_procfits,path_stats):
    # Clean up data
    move_reduced_science_data(path_rawfits,path_procfits,path_stats)
    compress_directory_fpack(path_rawfits)
    return None

def clean_up_common_data(data_path,path_rawbias,path_procfits,path_stats):
    # Clean up data
    move_reduced_bias_data(path_rawbias,path_procfits,path_stats)
    compress_directory_fpack(data_path)
    compress_directory_fpack(path_rawbias)
    if os.path.exists(path_procfits):
        compress_directory_fpack(path_procfits)
    return None


#def process_images(base_dir,data_dir,path_rawfits,path_rawbias,path_procfits,path_stats,instr_id,filter_id,reduction_log):
#    print('\n{:s} - Processing {:s}-band science data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),filter_id,data_dir))
#    reduction_log.write('\n{:s} - Processing {:s}-band science data in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),filter_id,data_dir))
#
#    flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename = find_flatfield_files(base_dir,data_dir,instr_id,filter_id)
#
#    if flatfield_chip1_filename != 'none' and flatfield_chip2_filename != 'none' and flatfield_chip3_filename != 'none':
#        if not os.path.isdir(path_procfits): os.mkdir(path_procfits)
#        if not os.path.isdir(path_stats):    os.mkdir(path_stats)
#        # Process raw science fits files
#        print('\n{:s} - Decompressing and splitting data files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
#        reduction_log.write('{:s} - Decompressing and splitting data files...\n')
#        os.chdir(path_rawfits)
#        decompress_directory_bzip2(path_rawfits)       # decompress downloaded GMOS files
#        decompress_directory_fpack(path_rawfits)       # decompress previously fpacked GMOS files
#        date_id = get_date_id()
#        create_stats_files(date_id,'scidata')    # create files to record image statistics
#        dim1, dim2 = split_extensions('scidata') # split science MEFs into individual FITS files
#        if instr_id == 'gmosn':
#            fix_gain_rdnoise_gmosn_hamamatsu()
#        elif instr_id == 'gmoss':
#            fix_gain_rdnoise_gmoss_hamamatsu()
#        compile_stats_files(date_id,'scidata')   # collect image stats of science files together and delete individual files
#
#        # decompress raw bias fits files
#        print('\n{:s} - Decompressing, filtering, and splitting bias files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
#        reduction_log.write('{:s} - Decompressing, filtering, and splitting bias files...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
#        os.chdir(path_rawbias)
#        decompress_directory_bzip2(path_rawbias)       # decompress downloaded GMOS files
#        decompress_directory_fpack(path_rawbias)       # decompress previously fpacked GMOS files
#        create_stats_files(date_id,'bias')       # create files to record image statistics
#        split_extensions('bias')                 # split bias MEFs into individual FITS files
#        if instr_id == 'gmosn':
#            fix_gain_rdnoise_gmosn_hamamatsu()
#        elif instr_id == 'gmoss':
#            fix_gain_rdnoise_gmoss_hamamatsu()
#        compile_stats_files(date_id,'bias')      # collect image stats of bias files together and delete individual files
#    
#        # perform overscan corrections and trim data for bias frames
#        os.chdir(path_rawbias)
#        overscan_and_trim()
#        bias_median_combine(date_id)
#
#        # Process science frames
#        os.chdir(path_rawfits)
#        overscan_and_trim()                                                    # Perform overscan correction and trim data
#        bias_correct(date_id,path_rawbias)                                      # Perform bias correction
#        concatenate_gmos_amps()                                                # Join science frames
#        flatfield_correction(base_dir,data_dir,flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename)  # Do flatfield corrections
#        cosmicray_cleaning()                                                   # Do cosmic ray cleaning
#        #fix_gmos_header_info(instr_id,filter_id)
#    
#        # Clean up data
#        move_reduced_data(path_rawfits,path_rawbias,path_procfits,path_stats)
#        #move_flatfield_images(base_dir,data_dir,path_procfits,flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename)
#        compress_directory_fpack(base_dir+data_dir)
#        compress_directory_fpack(path_rawfits)
#        compress_directory_fpack(path_rawbias)
#        compress_directory_fpack(path_procfits)
#        print('{:s} - Processing science data in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#        reduction_log.write('{:s} - Processing science data in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#        
#    else:
#        print('{:s} - Appropriate flatfield images for {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawfits))        
#        reduction_log.write('{:s} - Appropriate flatfield images for {:s} not found.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawfits))        
#
#    return None


In [ ]:
def process_gemini_science_directory(data_path,observatory,date_id):
    if observatory == 'GeminiN':   instr_id = 'gmosn'
    elif observatory == 'GeminiS': instr_id = 'gmoss'
    else: instr_id = 'not recognized'
    
    if instr_id == 'gmosn' or instr_id == 'gmoss':
        os.chdir(data_path)
        reduction_logfile = data_path + 'log_reduction_{:s}_{:s}_{:s}.txt'.format(date_id,instr_id,datetime.datetime.today().strftime('%Y%m%d_%H%M%S'))
        path_rawbias  = data_path + 'rawfits_bias/'
        path_procfits = data_path + 'flatfits_pyt/'
        path_stats    = data_path + 'stats/'
        with open(reduction_logfile,'w') as reduction_log:
            if os.path.exists(path_procfits):
                print('{:s} - Science data in {:s} already processed.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
            elif not os.path.exists(path_rawbias):
                print('{:s} - Bias directory in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
                reduction_log.write('{:s} - Bias directory in {:s} not found.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
            elif len(os.listdir(path_rawbias)) == 0:
                print('{:s} - Bias directory in {:s} is empty.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
                reduction_log.write('{:s} - Bias directory in {:s} is empty.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
            elif len(glob.glob('rawfits_?/')) == 0:
                print('{:s} - Raw science data folder(s) in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
                reduction_log.write('{:s} - Raw science data folder(s) in {:s} not found.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
            else:
                print('{:s} - Starting processing of data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
                reduction_log.write('{:s} - Starting processing of data in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
                for dir_rawfits in sorted(glob.glob('rawfits_?/')):
                    os.chdir(data_path + dir_rawfits)
                    date_id = get_date_id()
                reduction_log = process_bias_data(path_rawbias,instr_id,date_id,reduction_log)
                os.chdir(data_path)
                for dir_rawfits in sorted(glob.glob('rawfits_?/')):
                    filter_id = dir_rawfits[-2:-1]
                    path_rawfits  = data_path + dir_rawfits
                    reduction_log = process_science_data(data_path,path_rawfits,path_rawbias,path_procfits,path_stats,instr_id,filter_id,reduction_log)
                    clean_up_science_data(path_rawfits,path_procfits,path_stats)
                    #trim_gmoss_data_with_bad_amp(path_procfits)
                clean_up_common_data(data_path,path_rawbias,path_procfits,path_stats)
                print('{:s} - Processing of data in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
                reduction_log.write('{:s} - Processing of data in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
    return None

#def process_gemini_science_data(base_dir):
#    os.chdir(base_dir)
#    for data_dir in sorted(glob.glob('ut*_gemini?/')):
#    
#        if data_dir[-2:-1] == 'N':   instr_id = 'gmosn'
#        elif data_dir[-2:-1] == 'S': instr_id = 'gmoss'
#        else: instr_id = 'not recognized'
#
#        if instr_id == 'gmosn' or instr_id == 'gmoss':
#            os.chdir(base_dir + data_dir)
#            reduction_logfile = base_dir + data_dir + 'log_reduction_{:s}_{:s}_{:s}.txt'.format(data_dir[2:10],instr_id,datetime.datetime.today().strftime('%Y%m%d_%H%M%S'))
#            path_rawbias  = base_dir + data_dir + 'rawfits_bias/'
#            path_procfits = base_dir + data_dir + 'flatfits_pyt/'
#            path_stats    = base_dir + data_dir + 'stats/'
#            if not os.path.exists(path_procfits):
#                with open(reduction_logfile,'w') as reduction_log:
#                    if os.path.exists(path_rawbias):
#                        if len(os.listdir(path_rawbias)) > 0:
#                            if len(glob.glob('rawfits_?/')) > 0:
#                                print('\n{:s} - Starting processing of data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#                                reduction_log.write('\n{:s} - Starting processing of data in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#                                os.chdir(base_dir + data_dir)
#                                for dir_rawfits in sorted(glob.glob('rawfits_?/')):
#                                    os.chdir(base_dir + data_dir + dir_rawfits)
#                                    date_id = get_date_id()
#                                reduction_log = process_bias_data(path_rawbias,instr_id,date_id,reduction_log)
#                                os.chdir(base_dir + data_dir)
#                                for dir_rawfits in sorted(glob.glob('rawfits_?/')):
#                                    filter_id = dir_rawfits[-2:-1]
#                                    path_rawfits  = base_dir + data_dir + dir_rawfits
#                                    reduction_log = process_science_data(base_dir,data_dir,path_rawfits,path_rawbias,path_procfits,path_stats,instr_id,filter_id,reduction_log)
#                                    clean_up_science_data(path_rawfits,path_procfits,path_stats)
#                                    trim_gmoss_data_with_bad_amp(path_procfits)
#                                clean_up_common_data(base_dir,data_dir,path_rawbias,path_procfits,path_stats)
#                                print('\n{:s} - Processing of data in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#                                reduction_log.write('{:s} - Processing of data in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#                            else:
#                                print('{:s} - Raw science data in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#                                reduction_log.write('{:s} - Raw science data in {:s} not found.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#                        else:
#                            print('{:s} - Bias directory in {:s} is empty.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#                            reduction_log.write('{:s} - Bias directory in {:s} is empty.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#                    else:
#                        print('{:s} - Bias directory in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#                        reduction_log.write('{:s} - Bias directory in {:s} not found.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#            else:
#                print('{:s} - Science data in {:s} already processed.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#        else:
#            print('{:s} - Directory name format not recognized for {:s}.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
#    return None


In [ ]:
# date_id: date of observations in YYYYMMDD format
# observatory: 'GeminiN' or 'GeminiS'

process_gemini_science_directory(data_path,observatory,date_id)
